In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [36]:
column_names = ['time[s]', 'deflexion[mm]', 'force[kgf]']
raw_diamond = pd.read_excel("./tensile test raw data/raw_data_diamond_rubber_bands.xlsx", index_col=0)  
raw_diamond.attrs['name'] = 'Diamond'
raw_diamond_linear_range = [50,300]

raw_sannex = pd.read_excel("./tensile test raw data/raw_data_sannex_rubber_bands.xlsx", index_col=0)  
raw_sannex.attrs['name'] = 'Sannex'
raw_sannex_linear_range = [50,300]

# raw_orange_round = pd.read_excel("./tesile test raw data/raw_data_orange_round_rubber_band_100mm.xlsx", index_col=0)  
# raw_orange_round.attrs['name'] = 'orange_round_100mm'
# raw_orange_round_linear_range = [0,1000]

raw_orange_round = pd.read_excel("./tensile test raw data/raw_data_orange_round_rubber_band.xlsx", index_col=0)  
raw_orange_round.attrs['name'] = 'Orange round'
raw_orange_round_linear_range = [40,550]

raw_orange_rect = pd.read_excel("./tensile test raw data/raw_data_orange_rect_rubber_band.xlsx", index_col=0)  
raw_orange_rect.attrs['name'] = 'Orange rect'
raw_orange_rect_linear_range = [200,500]

raw_shortened_orange_round_single = pd.read_excel("./tensile test raw data/raw_data_shortened_orange_round_3.xlsx", index_col=0, usecols='A:D')  
raw_shortened_orange_round_single.attrs['name'] = 'Shortened orange round'
raw_shortened_orange_round_single_linear_range = [90,400]

raw_shortened_orange_round = []
raw_shortened_orange_round_linear_range = []
for i in range(5):
    raw_shortened_orange_round.append(pd.read_excel("./tensile test raw data/raw_data_shortened_orange_round_{}.xlsx".format(i+1), index_col=0, usecols='A:D'))
    raw_shortened_orange_round[i].attrs['name'] = 'Shortened orange round {}'.format(i+1)
    raw_shortened_orange_round_linear_range.append([50,400])

print(raw_shortened_orange_round)

[      时间( s)   变形( mm)   力( kgf) 
No.                              
1       0.000      0.00      0.00
2       0.092      0.00      0.00
3       0.127      0.00      0.00
4       0.155      0.00      0.00
5       0.185      0.00      0.00
...       ...       ...       ...
3235  116.419    299.98      2.23
3236  116.453    299.98      2.23
3237  116.490    299.99      2.23
3238  116.528    299.99      2.23
3239  116.595    300.00      2.23

[3239 rows x 3 columns],       时间( s)   变形( mm)   力( kgf) 
No.                              
1       0.000      0.00      0.00
2       0.118      0.00      0.00
3       0.152      0.00      0.00
4       0.187      0.00      0.00
5       0.216      0.00      0.00
...       ...       ...       ...
3070  113.894    299.98      2.08
3071  113.926    299.99      2.08
3072  113.969    299.99      2.08
3073  114.012    299.99      2.08
3074  114.050    300.00      2.08

[3074 rows x 3 columns],       时间( s)   变形( mm)   力( kgf) 
No.                          

# Comparison of all four types of rubber bands

In [14]:
%gui osx
%matplotlib
# %matplotlib inline

raw_rubber_bands = []
linear_ranges = []
raw_rubber_bands.extend([raw_diamond, raw_sannex, raw_orange_round,raw_orange_rect])
linear_ranges.extend([raw_diamond_linear_range,raw_sannex_linear_range,raw_orange_round_linear_range,raw_orange_rect_linear_range])

for i in raw_rubber_bands:
    i.columns = column_names
#     print('this is rubber band {}'.format(i.attrs['name']))
#     print(i.describe()

show_linear_reg = True
show_linear_range = True

figure, axes = plt.subplots(2, 2,figsize=[9.6, 7.2], dpi=100)
figure.suptitle('Force against deflexion for four types of rubber bands', size = 15, weight = 'semibold')
figure.tight_layout(pad = 3)
plt.subplots_adjust(left=0.08, bottom=.08, right=.96, top=.9)

for index in range(len(raw_rubber_bands)):
    df = raw_rubber_bands[index]
    print('this is rubber band {}'.format(df.attrs['name']))
    k = df[df['deflexion[mm]'].between(*linear_ranges[index], inclusive=False)]
    print(k)
    
    x = df['deflexion[mm]'].values.reshape(-1,1)
    y = df['force[kgf]'].values.reshape(-1,1)
    
    linear_x = k['deflexion[mm]'].values.reshape(-1, 1)
    linear_y = k['force[kgf]'].values.reshape(-1, 1)
    
    axis = plt.subplot(2,2,index+1)
    axis.set_title("{} rubber band".format(df.attrs['name']), color = 'maroon', weight = 'semibold')
    axis.set_ylabel("Force[kgf]")
    axis.set_xlabel('Deflexion[mm]')
    
    axis.set_xlim(0,700)
    axis.set_ylim(0,13)
    
    if show_linear_reg:
        linear_regressor = LinearRegression()  # create object for the class
        linear_regressor.fit(linear_x, linear_y)  # perform linear regression
        Y_pred = linear_regressor.predict(linear_x)  # make predictions
        print('slope is {}, intercept is {}'.format(*linear_regressor.coef_, *linear_regressor.intercept_))
        spring_rate = linear_regressor.coef_*9.806*1000
        print('spring rate:\n{}'.format(float(spring_rate)))
        display_text = 'Spring rate:\n{:.2f}[N/m]\n\n Linear range:\n{:.0f}[mm] to {:.0f}[mm]: \n{:.0f}[mm]'\
                        .format(float(spring_rate), 
                                float(linear_x[0]), 
                                float(linear_x[-1]), 
                                float(linear_x[-1]-linear_x[0]))
        axis.plot(linear_x, Y_pred, color='red')
        axis.text(x=(linear_x[0]+linear_x[-1])/2, y=0.5*(sum(axis.get_ylim())),\
                  s=display_text,\
                  transform=axis.transData,\
                  horizontalalignment='center')
        axis.set_xticks(list(axis.get_xticks()))
#         axis.set_xticks([float(linear_x[0]),float(linear_x[-1])])
#         axis.set_xticks(list(axis.get_xticks())+[linear_x[0], linear_x[-1]])
#         axis.text(x=0.5, y=0.5,s='spring rate is {}'.format(spring_rate),transform=axis.transAxes)


        
    
    if show_linear_range:
        axis.axvline(linear_x[0], color = 'red')
        axis.axvline(linear_x[-1], color = 'red')
    
    
    
    axis.scatter(x, y, s =1)

plt.show()

Using matplotlib backend: MacOSX
this is rubber band Diamond
      time[s]  deflexion[mm]  force[kgf]
No.                                     
774    24.941          50.04        0.65
775    24.974          50.16        0.65
776    25.010          50.31        0.65
777    25.041          50.36        0.65
778    25.069          50.44        0.65
...       ...            ...         ...
3117   99.712         299.45        2.41
3118   99.744         299.57        2.41
3119   99.779         299.70        2.41
3120   99.812         299.81        2.41
3121   99.842         299.89        2.41

[2348 rows x 3 columns]
slope is [0.0069659], intercept is 0.2295549920468869
spring rate:
68.30759162143697
this is rubber band Sannex
      time[s]  deflexion[mm]  force[kgf]
No.                                     
684    21.302          50.05        0.54
685    21.333          50.19        0.54
686    21.363          50.29        0.55
687    21.399          50.40        0.55
688    21.432          

# Comparison of the original and the shortened orange round rubber band

In [21]:
%gui osx
%matplotlib
# %matplotlib inline

raw_rubber_bands = []
linear_ranges = []
raw_rubber_bands.extend([raw_orange_round,raw_shortened_orange_round])
linear_ranges.extend([raw_orange_round_linear_range,raw_shortened_orange_round_linear_range])

for i in raw_rubber_bands:
    i.columns = column_names
#     print('this is rubber band {}'.format(i.attrs['name']))
#     print(i.describe()

show_linear_reg = True
show_linear_range = True

figure, axes = plt.subplots(1, 2,figsize=[9.6, 4.8], dpi=100)
figure.suptitle('Force against deflexion for the original and \nshortend orange round rubber band', size = 15, weight = 'semibold')
figure.tight_layout(pad = 3)
plt.subplots_adjust(left=0.08, bottom=.15, right=.96, top=.8)

for index in range(len(raw_rubber_bands)):
    df = raw_rubber_bands[index]
    print('this is rubber band {}'.format(df.attrs['name']))
    k = df[df['deflexion[mm]'].between(*linear_ranges[index], inclusive=False)]
    print(k)
    
    x = df['deflexion[mm]'].values.reshape(-1,1)
    y = df['force[kgf]'].values.reshape(-1,1)
    
    linear_x = k['deflexion[mm]'].values.reshape(-1, 1)
    linear_y = k['force[kgf]'].values.reshape(-1, 1)
    
    axis = plt.subplot(1,2,index+1)
    axis.set_title("{} rubber band".format(df.attrs['name']), color = 'maroon', weight = 'semibold')
    axis.set_ylabel("Force[kgf]")
    axis.set_xlabel('Deflexion[mm]')
    
    axis.set_xlim(0,700)
    axis.set_ylim(0,13)
    
    if show_linear_reg:
        linear_regressor = LinearRegression()  # create object for the class
        linear_regressor.fit(linear_x, linear_y)  # perform linear regression
        Y_pred = linear_regressor.predict(linear_x)  # make predictions
        print('slope is {}, intercept is {}'.format(*linear_regressor.coef_, *linear_regressor.intercept_))
        spring_rate = linear_regressor.coef_*9.806*1000
        print('spring rate:\n{}'.format(float(spring_rate)))
        display_text = 'Spring rate:\n{:.2f}[N/m]\n\n Linear range:\n{:.0f}[mm] to {:.0f}[mm]: \n{:.0f}[mm]'\
                        .format(float(spring_rate), 
                                float(linear_x[0]), 
                                float(linear_x[-1]), 
                                float(linear_x[-1]-linear_x[0]))
        axis.plot(linear_x, Y_pred, color='red')
        axis.text(x=(linear_x[0]+linear_x[-1])/2, y=0.5*(sum(axis.get_ylim())),\
                  s=display_text,\
                  transform=axis.transData,\
                  horizontalalignment='center')

    if show_linear_range:
        axis.axvline(linear_x[0], color = 'red')
        axis.axvline(linear_x[-1], color = 'red')
    
    
    
    axis.scatter(x, y, s =1)

plt.show()

Using matplotlib backend: MacOSX
this is rubber band Orange round
      time[s]  deflexion[mm]  force[kgf]
No.                                     
573    17.864          40.08        0.49
574    17.892          40.18        0.49
575    17.924          40.31        0.49
576    17.954          40.39        0.49
577    17.989          40.51        0.49
...       ...            ...         ...
5356  170.583         549.50        3.09
5357  170.613         549.63        3.09
5358  170.646         549.74        3.09
5359  170.677         549.84        3.09
5360  170.713         549.95        3.09

[4788 rows x 3 columns]
slope is [0.00483999], intercept is 0.2424856679412808
spring rate:
47.4609047228068
this is rubber band Shortened orange round
      time[s]  deflexion[mm]  force[kgf]
No.                                     
1381   50.508          90.09        0.91
1382   50.540          90.16        0.91
1383   50.576          90.31        0.91
1384   50.618          90.47        0.92
13

In [67]:
%gui osx
%matplotlib
# %matplotlib inline

raw_rubber_bands = raw_shortened_orange_round
linear_ranges = raw_shortened_orange_round_linear_range
# raw_rubber_bands.extend([raw_orange_round,raw_shortened_orange_round])
# linear_ranges.extend([raw_orange_round_linear_range,raw_shortened_orange_round_linear_range])

for i in raw_rubber_bands:
    i.columns = column_names
#     print('this is rubber band {}'.format(i.attrs['name']))
#     print(i.describe()

show_linear_reg = True
show_linear_range = True

spring_rates = []

figure, axes = plt.subplots(2, 3,figsize=[9.6, 7.2], dpi=100)
figure.suptitle('Force against deflexion for 5 shortend orange round rubber bands', size = 15, weight = 'semibold')
figure.tight_layout(pad = 3)
plt.subplots_adjust(left=0.04, bottom=.08, right=.96, top=.9)

for index in range(len(raw_rubber_bands)):
    print('length is {}'.format(len(raw_rubber_bands)))
    df = raw_rubber_bands[index]
    print('this is rubber band {}'.format(df.attrs['name']))
    k = df[df['deflexion[mm]'].between(*linear_ranges[index], inclusive=False)]
    print(k)
    
    x = df['deflexion[mm]'].values.reshape(-1,1)
    y = df['force[kgf]'].values.reshape(-1,1)
    
    linear_x = k['deflexion[mm]'].values.reshape(-1, 1)
    linear_y = k['force[kgf]'].values.reshape(-1, 1)
    
    axis = plt.subplot(2,3,index+1)
    axis.set_title("{} rubber band".format(df.attrs['name']), color = 'maroon', weight = 'semibold')
    axis.set_ylabel("Force[kgf]")
    axis.set_xlabel('Deflexion[mm]')
    
    axis.set_xlim(0,400)
    axis.set_ylim(0,13)
    
    if show_linear_reg:
        linear_regressor = LinearRegression()  # create object for the class
        linear_regressor.fit(linear_x, linear_y)  # perform linear regression
        Y_pred = linear_regressor.predict(linear_x)  # make predictions
        print('slope is {}, intercept is {}'.format(*linear_regressor.coef_, *linear_regressor.intercept_))
        spring_rate = linear_regressor.coef_*9.806*1000
        spring_rates.append(float(spring_rate))
        print('spring rate:\n{}'.format(float(spring_rate)))
        display_text = 'Spring rate:\n{:.2f}[N/m]\n\n Linear range:\n{:.0f}[mm] to {:.0f}[mm]: \n{:.0f}[mm]'\
                        .format(float(spring_rate), 
                                float(linear_x[0]), 
                                float(linear_x[-1]), 
                                float(linear_x[-1]-linear_x[0]))
        axis.plot(linear_x, Y_pred, color='red')
        axis.text(x=(linear_x[0]+linear_x[-1])/2, y=0.5*(sum(axis.get_ylim())),\
                  s=display_text,\
                  transform=axis.transData,\
                  horizontalalignment='center')

    if show_linear_range:
        axis.axvline(linear_x[0], color = 'red')
        axis.axvline(linear_x[-1], color = 'red')
    
    
    
    axis.scatter(x, y, s =1)
axis = plt.subplot(2,3, 6)

spring_df_dict = pd.DataFrame(spring_rates)
mean = spring_df_dict.mean()
median = spring_df_dict.median()

# # print(spring_df.describe().to_dict())
# sum_stat = ''
# for key, value in spring_df_dict[0]:
#     sum_stat+='{}:{}\n'.format(key,value)
axis.text(0.5, 0.5, transform=axis.transData,\
         horizontalalignment='center',\
         verticalalignment='center',\
         s='mean: {:.2f}\n median: {:.2f}'.format(mean[0], median[0]))
axis.set_title("Summary statistics", color = 'maroon', weight = 'semibold')
plt.show()

Using matplotlib backend: MacOSX
length is 5
this is rubber band Shortened orange round 1
      time[s]  deflexion[mm]  force[kgf]
No.                                     
1050   38.396          50.09        0.67
1051   38.429          50.20        0.67
1052   38.465          50.34        0.67
1053   38.511          50.47        0.67
1054   38.549          50.60        0.67
...       ...            ...         ...
3235  116.419         299.98        2.23
3236  116.453         299.98        2.23
3237  116.490         299.99        2.23
3238  116.528         299.99        2.23
3239  116.595         300.00        2.23

[2190 rows x 3 columns]
slope is [0.00622684], intercept is 0.3062888312324956
spring rate:
61.0603755090914
length is 5
this is rubber band Shortened orange round 2
      time[s]  deflexion[mm]  force[kgf]
No.                                     
963    35.850          50.03        0.64
964    35.886          50.13        0.65
965    35.922          50.28        0.65
966  